<a href="https://colab.research.google.com/github/DaniFarias/Previsao_de_Tensoes_TCC_FURG/blob/main/Previs%C3%A3o_de_Tens%C3%B5es_TCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
# Importa bibliotecas

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn

In [ ]:
# Importa do dataset
tensoes = pd.read_csv('TensaoOriginal.csv', thousands=',', delimiter=';', encoding='latin1', na_values='n/a')
tensoes.head(3)

In [ ]:
#list(tensoes)
#tensoes.info()
#tensoes.describe()

In [4]:
# Drop coluna Tempo
tensoes.drop(columns=['Tempo'], inplace = True)
tensoes.head(3)

In [5]:
# Para garantir um resultado idêntico, iniciamos a geração randomica com a mesma semente (seed)
np.random.seed(42)

In [6]:
# Split dos dados de treino e teste
from sklearn.model_selection import train_test_split

tensoes_treino, tensoes_teste = train_test_split(tensoes, 
                                                 test_size=0.2, 
                                                 random_state=42)
print('tensoes_treino:', len(tensoes_treino))
print('tensoes_teste: ', len(tensoes_teste))

tensoes_treino: 171752
tensoes_teste:  42938


In [7]:
# Separar treino
X = tensoes_treino.drop('Tensao12', axis=1)
y = tensoes_treino['Tensao12'].copy()

In [36]:
tensoes.sample(50)

,Corrente,Cap1,Cap2,Cap3,Cap4,Cap5,Cap6,Cap7,Cap8,Cap9,...,Tensao3,Tensao4,Tensao5,Tensao6,Tensao7,Tensao8,Tensao9,Tensao10,Tensao11,Tensao12
19161,348.79,0,0,0,0,1,1,1,1,0,...,20248.05,20253.78,20077.66,19822.19,19594.66,19421.94,19549.83,19796.56,19990.95,20144.19
86267,433.85,0,0,0,0,0,0,0,0,1,...,18707.45,18855.55,18859.42,18933.80,18955.53,19051.79,18624.93,18175.41,18307.11,18445.62
201635,-180.65,1,1,1,1,1,1,1,1,0,...,21338.85,21397.70,21468.29,21596.75,21652.74,21547.24,21197.76,21137.72,21080.03,18381.54
9510,485.89,0,1,1,0,0,0,0,0,0,...,19028.12,18846.94,18854.06,18972.40,19031.47,19151.23,19251.43,19325.77,19480.62,19570.20
62419,397.54,0,0,0,0,0,1,1,0,0,...,19968.14,20051.78,20079.96,19838.75,19544.26,19395.81,19424.47,19526.36,19594.79,19659.61
120822,453.01,0,0,0,0,1,1,0,0,0,...,20824.19,20808.10,20487.82,20043.06,20292.00,20565.29,20777.86,20864.17,20857.90,18940.86
12282,-76.26,1,1,1,1,1,1,1,1,0,...,20423.34,20390.44,20431.75,20546.28,20626.33,20716.39,20792.18,20821.98,20704.77,20657.93
33361,235.15,0,1,1,1,1,1,0,0,0,...,18241.97,18089.38,17842.78,17694.11,17745.18,17903.82,18000.32,18117.42,18373.36,18465.89
50146,46.35,1,1,1,1,1,0,0,0,0,...,17748.19,17837.63,17775.92,17778.95,17844.50,17854.69,17869.42,17837.95,17975.39,17882.26
109187,-136.90,1,1,1,1,1,1,1,1,1,...,18961.70,18999.09,18913.70,18889.28,18862.64,18918.99,18964.14,18920.07,19002.43,19027.49


# 1. Modelo Arvore de Decisão

In [8]:
# Criação e treinamento do modelo por Arvore de Decisão
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=42, 
                                 max_depth=10)
tree_reg.fit(X,y)

DecisionTreeRegressor(max_depth=10, random_state=42)

In [ ]:
#print('Acurácia: {:.5%}'.format(tree_reg.score(X, y)))

In [12]:
# Usando a medida de desempenho RMSE
from sklearn.metrics import mean_squared_error

tensoes_predictions = tree_reg.predict(X)

tree_mse = mean_squared_error(y, tensoes_predictions)
tree_rmse = np.sqrt(tree_mse)
print("RMSE:",tree_rmse)

RMSE: 112.17645060558498


In [11]:
# Validação Cruzada para o modelo de Árvore de Decisão
from sklearn.model_selection import cross_val_score

tree_scores = cross_val_score(tree_reg, X, y
                          ,scoring='neg_mean_squared_error'
                          ,cv=5)
tree_scores = np.sqrt(-tree_scores)
print("Mean:  ", tree_scores.mean())
print("Std:   ", tree_scores.std())

Mean:   112.94750705900552
Std:    4.75655878868191


# 2. Floresta Aleatória

In [13]:
# Aqui um modelo de Floresta Aleatória -- demora um pouco mais que os anteriores
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=30
                                   ,random_state=42)
forest_reg.fit(X, y)

RandomForestRegressor(n_estimators=30, random_state=42)

In [14]:
# Usando a medida de desempenho RMSE
from sklearn.metrics import mean_squared_error

tensoes_predictions = forest_reg.predict(X)

forest_mse = mean_squared_error(y, tensoes_predictions)
forest_rmse = np.sqrt(forest_mse)
print("RMSE:", forest_rmse)

RMSE: 2.6200649689198405


In [22]:
# Validação cruzada para o modelo de Floresta Aleatória
from sklearn.model_selection import cross_val_score

scores = cross_val_score(forest_reg, X, y
                         ,scoring='neg_mean_squared_error'
                         ,cv=10)
scores = np.sqrt(-scores)
print("Mean:  ", scores.mean())
print("Std:   ", scores.std())

Mean:   6.583189203841551
Std:    1.0818084910929295


In [16]:
# Exibe resultados
print("Scores:", scores)
print("Mean:  ", scores.mean())
print("Std:   ", scores.std())

Scores: [7.09212194 8.23392306 6.40110274 5.92452887 6.55763265 5.7076695
 7.62143826 5.43499228 8.01689173 4.841591  ]
Mean:   6.583189203841551
Std:    1.0818084910929295


In [17]:
# Separar teste
X_test = tensoes_teste.drop('Tensao12', axis=1)
y_test = tensoes_teste['Tensao12'].copy()

In [18]:
# Usando a medida de desempenho RMSE para dados de Teste
final_predictions = forest_reg.predict(X_test)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
print("RMSE: ",(final_rmse))

Erro:  7.230025757633563


In [19]:
# Usando a medida de desempenho CV para dados de Teste
scores = cross_val_score(forest_reg, X, y, 
                         scoring='neg_mean_squared_error', cv=10)
scores = np.sqrt(-scores)

In [20]:
print("Scores:", scores)
print("Mean:  ", scores.mean())
print("Std:   ", scores.std())

Scores: [7.09212194 8.23392306 6.40110274 5.92452887 6.55763265 5.7076695
 7.62143826 5.43499228 8.01689173 4.841591  ]
Mean:   6.583189203841551
Std:    1.0818084910929295


In [ ]:
#Classificação de Features
forest_reg.feature_importances_

for score, name in sorted(zip(forest_reg.feature_importances_, tensoes)):
    print('{:.2%}'.format(score), name)

In [27]:
#print('Acurácia: {:.5%}'.format(forest_reg.score(X_test, y_test)))

In [24]:
z = tensoes_predictions.tolist()

In [ ]:
# Comparação tensoes de teste e preditas
pd.options.display.float_format = '{:.2f}'.format
df = pd.DataFrame((zip(y, z)), columns = ['T_12', 'T_12_predict'])
df.sample(50)